In [1]:
import os
import json
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from IPython.core.display import display, HTML
from imagenet_info import imagenet_info
from imagenet import imagenet_info, imagenet_train, imagenet_valid
from Darknet53_model import Darknet53
import datetime

In [2]:
os.environ["CUDA_VISIBLE_DIVICES"] = "14"
gpus = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(gpus[0], True)
display(HTML("<style>.container { width:98% !important;}</style>"))

## Build Dataset

In [3]:
imgnet_info  = imagenet_info("map_clsloc.txt", "ILSVRC2012_validation_ground_truth.txt")
imgnet_train = imagenet_train("/raid/peterchen/datasets/imagenet/2012/train/", 256, imgnet_info, batch_size=256)
imgnet_valid = imagenet_valid("/raid/peterchen/datasets/imagenet/2012/valid/", 256, imgnet_info, batch_size=250)
imgnet_train_dist_dataset = strategy.experimental_distribute_dataset(imgnet_train.Dateset)
imgnet_valid_dist_dataset = strategy.experimental_distribute_dataset(imgnet_valid.Dateset)

## Build Model

In [4]:
model = Darknet53()

## Setting

In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam(0.0001)

In [6]:
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")
valid_loss = tf.keras.metrics.Mean(name="valid_loss")
valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="valid_accuracy")

In [7]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [8]:
@tf.function
def valid_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=False)
        loss = loss_object(labels, predictions)
    valid_loss(loss)
    valid_accuracy(labels, predictions)

## Training

In [9]:
time = datetime.datetime.now()
#weight_name = "weights"+'_'+str(time.month)+'-'+str(time.day)+'/'
weight_name = "weights.h5"
weights_path = os.path.join("weights", weight_name)
#if not os.path.exists(weights_path):
#    os.mkdir(weights_path)

In [12]:
EPOCHS = 30

In [13]:
best_loss = np.inf
print("Training Start!")
for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    valid_loss.reset_states()
    valid_accuracy.reset_states()
    
    count = 0
    for images, labels in imgnet_train.Dataset:
        train_step(images, labels)
        count += 1
        if count > 1000:
            break

    count = 0
    for images, labels in imgnet_valid.Dataset:
        valid_step(images, labels)
        count += 1
        if count > 200:
            break
    
    template = "Epoch {}, Loss: {:4.3f}, Accuracy: {:4.3f}, Valid Loss: {:4.3f}, Valid Accuracy: {:4.3f}"
    print(template.format(epoch+1, train_loss.result(), train_accuracy.result(),
                          valid_loss.result(), valid_accuracy.result()))
    
    if valid_loss.result().numpy() < best_loss:
        best_loss = valid_loss.result().numpy()
        model.save_weights(weights_path)
        print("Model saved! with loss: {:4.3f}".format(valid_loss.result().numpy()))
        print("")
        
print("Training End!")

Training Start!


KeyboardInterrupt: 